In [ ]:
PROJECT_ID = 'vijays-sandbox'
BUCKET = 'vijays-sandbox-ml'
MODEL_PATH = 'taxi_trained_dnn/5000/export/exporter/1564424807'
MODEL_NAME = 'taxifare'
VERSION_NAME = 'v1'

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'service_account_key.json' # for local ONLY
os.environ["BUCKET"] = BUCKET

# Deploy for Online Prediction

To get our predictions we need to fetch the latest traffic information from BigQuery. Only then can we invoke our tensorflow model.

To do this we'll take advantage of [AI Platforms Custom Prediction Routines](https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routines) which allows us to execute custom python code in response to every online prediction request. There are 5 steps to creating a custom prediction routine:

1. Upload Model Artifacts to GCS
2. Implement Predictor interface 
3. Package the prediction code and dependencies
4. Deploy
5. Invoke API

## 1. Upload Model Artifacts to GCS

Here we upload our model weights so that AI Platform can access them.

In [ ]:
!gsutil cp -r $MODEL_PATH/* gs://$BUCKET/taxifare/model/

## 2. Implement Predictor Interface

Interface Spec: https://cloud.google.com/ml-engine/docs/tensorflow/custom-prediction-routines#predictor-class

This tells AI Platform how to load the model artifacts, and is where we specify our custom prediction code.

In [ ]:
%%writefile predictor.py
import tensorflow as tf
from google.cloud import bigquery

class TaxifarePredictor(object):
    def __init__(self, predict_fn):
      self.predict_fn = predict_fn
    
    
    def predict(self, instances, **kwargs):
        bq = bigquery.Client()
        query_string = """
        SELECT
          *
        FROM
          `vijays-sandbox.taxifare.traffic_realtime`
        ORDER BY
          time DESC
        LIMIT 1
        """
        trips = bq.query(query_string).to_dataframe()['trips_last_5min'][0]
        instances['trips_last_5min'] = [trips for _ in range(len(instances.items()[0][1]))]
        predictions = self.predict_fn(instances)
        return predictions
    

    @classmethod
    def from_path(cls, model_dir):
        predict_fn = tf.contrib.predictor.from_saved_model(model_dir,'predict')
        return cls(predict_fn)

### Test Predictor Class Works Locally

In [ ]:
import predictor

instances = {'dayofweek' : [6,7],
             'hourofday' : [12,11],
             'pickuplon' : [-73.99,-73.99], 
             'pickuplat' : [40.758,40.758],
             'dropofflat' : [40.742,40.758],
             'dropofflon' : [-73.97,-73.97]}

predictor = predictor.TaxifarePredictor.from_path(MODEL_PATH)
predictor.predict(instances)

## 3. Package Predictor Class and Dependencies

We must package the predictor as a tar.gz source distribution package.

In [ ]:
%%writefile setup.py
from setuptools import setup

setup(
    name='taxifare_custom_predict_code',
    version='0.1',
    scripts=['predictor.py'])

In [ ]:
!python setup.py sdist --formats=gztar

In [ ]:
!gsutil cp dist/taxifare_custom_predict_code-0.1.tar.gz gs://$BUCKET/taxifare/predict_code/

## 4. Deploy

This is similar to how we deploy standard models to AI Platform, with a few extra command line arguments.

*Warning: If you get a GCS access error, grant the 'Storage Object Viewer' role on the bucket that contains your artifacts to the service account being used.*

In [ ]:
#!gcloud ai-platform models create $MODEL_NAME --regions us-central1 --enable-logging

!gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --origin gs://$BUCKET/taxifare/model \
  --runtime-version 1.14 \
  --python-version 2.7 \
  --package-uris gs://$BUCKET/taxifare/predict_code/taxifare_custom_predict_code-0.1.tar.gz \
  --prediction-class predictor.TaxifarePredictor

In [ ]:
%%bash
echo Granting $SVC_ACCOUNT read access to gs://$BUCKET
PROJECT_ID=$(gcloud config list project --format "value(core.project)")
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print response['serviceAccount']")
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET

## 5. Invoke API

We will use the google api python client. You can install with 

`pip install --upgrade google-api-python-client`

In [ ]:
import googleapiclient.discovery

instances = {'dayofweek' : [6], 
             'hourofday' : [12],
             'pickuplon' : [-73.99], 
             'pickuplat' : [40.758],
             'dropofflat' : [40.742],
             'dropofflon' : [-73.97]}

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)

response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])